# Few-Shot Prompting 기법 가이드

## 개요

**Few-Shot Prompting**은 AI 언어 모델에게 소수의 예시를 제공하여 특정 작업의 성능을 향상시키는 프롬프트 엔지니어링 기법입니다. 이 기법은 모델이 원하는 출력 형식과 작업 패턴을 학습할 수 있도록 구체적인 예시를 통해 가이드를 제공합니다.

### Few-Shot Prompting의 작동 원리

Few-Shot Prompting은 다음과 같은 구조로 작동합니다:

| 구성 요소 | 설명 | 예시 |
|----------|------|------|
| **Task Description** | 수행할 작업에 대한 설명 | "다음 질문에 단계적으로 답변하세요" |
| **Examples** | 입력-출력 쌍의 예시들 | 질문 → 단계별 추론 과정 |
| **Query** | 실제 처리할 새로운 입력 | 사용자의 실제 질문 |

### 기대 효과

- **정확성 향상**: 예시를 통한 명확한 출력 형식 지정
- **일관성 확보**: 동일한 형태의 결과 생성
- **복잡한 작업 처리**: 단순 지시보다 정교한 작업 수행 가능
- **맥락 이해**: 작업의 의도와 목적에 대한 더 나은 이해

## 주요 학습 내용

### 1. FewShotPromptTemplate 기본 사용법
- 예시 데이터 구성 방법
- 프롬프트 템플릿 생성 및 활용
- 체인 구성을 통한 실행

### 2. Example Selector를 통한 동적 예시 선택
- 유사성 기반 예시 자동 선택
- 벡터 데이터베이스를 활용한 효율적인 예시 관리
- 상황에 따른 최적 예시 적용

### 3. FewShotChatMessagePromptTemplate
- 대화형 Few-Shot 프롬프팅
- 채팅 형태의 예시 구성
- 다양한 작업 유형별 적용 방법

### 기술적 요구사항

| 항목 | 요구사항 |
|------|---------|
| **Python 패키지** | langchain-core, langchain-openai, langchain-chroma |
| **API** | OpenAI API 또는 호환 가능한 LLM API |
| **벡터 데이터베이스** | Chroma (예시 선택 기능용) |
| **임베딩 모델** | text-embedding-3-small |

In [14]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

True

In [15]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangChain-Tutorial


In [16]:
import os
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response

# Create ChatOpenAI object
llm = ChatOpenAI(
    temperature=0.1,
    model_name="openai/gpt-4.1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)

# Simple question to test LLM
question = "대한민국의 수도는 뭐야?"

# Execute query
answer = llm.stream(question)
stream_response(answer)

대한민국의 수도는 서울특별시(서울)입니다.

In [17]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Few-Shot에 사용할 예시 데이터 정의 (복잡한 추론 과정을 단계별로 보여주는 예시들)
examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

In [18]:
# 개별 예시를 위한 프롬프트 템플릿 생성
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

# 첫 번째 예시로 템플릿 테스트
print(example_prompt.format(**examples[0]))

Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인



In [19]:
# FewShotPromptTemplate 생성 (모든 예시를 포함)
prompt = FewShotPromptTemplate(
    examples=examples,  # 사용할 예시 리스트
    example_prompt=example_prompt,  # 개별 예시의 형식
    suffix="Question:\n{question}\nAnswer:",  # 실제 질문이 들어갈 부분
    input_variables=["question"],  # 입력 변수 정의
)

# 새로운 질문으로 프롬프트 생성
question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인


Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일


Question:
율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군


Question:
올드보이와 기생충의 감독이 같은 나라 출신인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예


Question:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?
Answer:


In [20]:
# LLM으로 Few-Shot 프롬프트 실행
answer = llm.stream(final_prompt)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google이 창립된 연도는 언제인가요?  
중간 답변: Google은 1998년에 창립되었습니다.  
추가 질문: Bill Gates는 언제 태어났나요?  
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.  
추가 질문: 1998년에 Bill Gates의 나이는 몇 살이었나요?  
중간 답변: 1998년 - 1955년 = 43년. Bill Gates는 1998년에 43세였습니다.  
최종 답변은: 43세

In [21]:
# FewShotPromptTemplate으로 LangChain chain 구성
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# Chain 생성: 프롬프트 → LLM → 출력 파서
chain = prompt | llm | StrOutputParser()

# Chain으로 질문 실행
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google이 창립된 연도는 언제인가요?  
중간 답변: Google은 1998년에 창립되었습니다.  
추가 질문: Bill Gates는 언제 태어났나요?  
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.  
추가 질문: 1998년에 Bill Gates의 나이는 몇 살이었나요?  
중간 답변: 1998년에서 1955년을 빼면 43이므로, Bill Gates는 1998년에 43세였습니다.  
최종 답변은: 43세

---

## Example Selector - 동적 예시 선택 시스템

### 개요

예시가 많아질수록 적절한 예시 선택이 중요해집니다. 모든 예시를 포함하면 프롬프트가 과도하게 길어지고, 관련 없는 예시는 AI의 성능을 저하시킬 수 있습니다.

**Example Selector**는 현재 쿼리와 가장 관련성이 높은 예시들을 자동으로 선택하는 시스템입니다.

### Example Selector의 작동 원리

| 단계 | 설명 | 기술적 구현 |
|------|------|-------------|
| **1. 예시 저장** | 모든 예시를 벡터 데이터베이스에 저장 | 임베딩을 통한 벡터 변환 |
| **2. 쿼리 분석** | 입력 쿼리를 벡터로 변환 | 동일한 임베딩 모델 사용 |
| **3. 유사성 계산** | 코사인 유사도 등을 통한 거리 측정 | 벡터 간 수치적 비교 |
| **4. 예시 선택** | 상위 k개의 가장 유사한 예시 선별 | 임계값 또는 개수 기준 |
| **5. 프롬프트 구성** | 선별된 예시들로 최적 프롬프트 생성 | 동적 템플릿 구성 |

### Example Selector 사용의 이점

#### 성능 측면
- **정확도 향상**: 맥락에 맞는 예시만 제공
- **응답 일관성**: 유사한 작업에 대한 일관된 패턴 학습
- **처리 효율성**: 불필요한 예시 제외로 빠른 처리

#### 비용 및 관리 측면  
- **토큰 최적화**: 관련 없는 예시 제외로 API 비용 절약
- **확장성**: 새로운 예시 추가 시 자동 최적화
- **유지보수**: 수동 예시 관리 부담 감소

### 주요 Example Selector 유형

| 유형 | 특징 | 적용 상황 |
|------|------|----------|
| **SemanticSimilarityExampleSelector** | 의미적 유사성 기반 | 일반적인 텍스트 작업 |
| **MaxMarginalRelevanceExampleSelector** | 다양성과 관련성 균형 | 다양한 예시가 필요한 경우 |
| **LengthBasedExampleSelector** | 길이 기반 선택 | 토큰 제한이 있는 환경 |

- [API 문서 참조](https://api.python.langchain.com/en/latest/core/example_selectors.html)

In [23]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# 벡터 데이터베이스 생성 (예시들을 벡터로 저장하기 위함)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("EMBEDDING_BASE_URL"),
)

chroma = Chroma("example_selector", embeddings)

In [31]:
# 유사성 기반 예시 선택기 생성
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 선택 가능한 예시 목록
    examples,
    # 의미적 유사성을 측정하는 임베딩 모델
    embeddings,
    # 임베딩을 저장하고 유사성 검색을 수행하는 벡터스토어
    chroma,
    # 선택할 예시의 개수
    k=1,
)

# 테스트 질문
question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

# 입력과 가장 유사한 예시 선택
selected_examples = example_selector.select_examples({"question": question})

print(f"입력에 가장 유사한 예시:\n{question}\n")
for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')

AuthenticationError: Error code: 401 - {'error': {'message': 'Your authentication token is not from a valid issuer.', 'type': 'invalid_request_error', 'code': 'invalid_issuer', 'param': None}}

In [33]:
# Example Selector를 사용하는 FewShotPromptTemplate 생성
prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # 예시 리스트 대신 선택기 사용
    example_prompt=example_prompt,  # 개별 예시의 형식
    suffix="Question:\n{question}\nAnswer:",  # 실제 질문 부분
    input_variables=["question"],  # 입력 변수
)

# 질문으로 프롬프트 생성 (자동으로 유사한 예시 선택됨)
question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
example_selector_prompt = prompt.format(question=question)
print(example_selector_prompt)

NameError: name 'example_selector' is not defined

### Example Selector와 FewShotPromptTemplate 통합

**Example Selector**를 **FewShotPromptTemplate**과 연결하여 동적 예시 선택 기능을 구현할 수 있습니다.

#### 구현 방법

**기본 방식 (정적 예시)**:
```python
FewShotPromptTemplate(
    examples=examples,  # 모든 예시를 포함
    example_prompt=example_prompt,
    suffix="Question: {question}\nAnswer:",
    input_variables=["question"]
)
```

**Example Selector 활용 (동적 예시)**:
```python
FewShotPromptTemplate(
    example_selector=example_selector,  # 동적 선택기 사용
    example_prompt=example_prompt,
    suffix="Question: {question}\nAnswer:",
    input_variables=["question"]
)
```

#### 주요 차이점

| 특성 | 정적 예시 방식 | 동적 선택 방식 |
|------|-------------|-------------|
| **예시 선택** | 모든 예시 포함 | 관련성 기반 자동 선택 |
| **프롬프트 길이** | 고정 (예시 개수에 비례) | 가변 (선택된 개수만큼) |
| **성능** | 예시가 많으면 성능 저하 가능 | 최적화된 성능 |
| **비용** | 높음 (많은 토큰 사용) | 효율적 (필요한 토큰만 사용) |

이 방식을 통해 **질문이 바뀔 때마다 가장 적절한 예시들이 자동으로 선택**되어 프롬프트에 포함됩니다.

In [32]:
# Example Selector가 적용된 FewShotPromptTemplate으로 체인 생성
prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # 동적 예시 선택기
    example_prompt=example_prompt,  # 개별 예시 형식
    suffix="Question:\n{question}\nAnswer:",  # 질문 템플릿
    input_variables=["question"],
)

# 체인 생성 (프롬프트 → LLM)
chain = prompt | llm

NameError: name 'example_selector' is not defined

In [34]:
# Example Selector를 사용한 체인 실행
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google이 창립된 연도는 언제인가요?  
중간 답변: Google은 1998년에 창립되었습니다.  
추가 질문: Bill Gates는 언제 태어났나요?  
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.  
추가 질문: 1998년에 Bill Gates의 나이는 몇 살이었나요?  
중간 답변: 1998년에서 1955년을 빼면 43이므로, Bill Gates는 1998년에 43세였습니다.  
최종 답변은: 43세

---

## FewShotChatMessagePromptTemplate - 대화형 Few-Shot

### 개요

**FewShotChatMessagePromptTemplate**은 대화형 인터페이스에 최적화된 Few-Shot 프롬프팅 기법입니다. 기존의 단순 텍스트 형태와 달리, Human과 AI 간의 대화 형태로 예시를 구성하여 더 자연스러운 상호작용을 가능하게 합니다.

### 텍스트 기반 vs 대화 기반 Few-Shot 비교

| 특성 | 텍스트 기반 | 대화 기반 |
|------|-------------|----------|
| **구조** | Input → Output | Human → AI |
| **맥락 이해** | 제한적 | 대화 맥락 고려 |
| **적용 분야** | 일반적인 변환 작업 | 챗봇, 상담 시스템 |
| **톤/스타일** | 형식적 | 자연스러운 대화체 |

### 대화형 Few-Shot의 장점

#### 맥락적 일관성
- **역할 인식**: AI가 대화 상대방으로서의 역할을 명확히 이해
- **톤 유지**: 예시에서 보여준 말투와 스타일 지속
- **대화 흐름**: 자연스러운 질문-답변 패턴 학습

#### 특수 상황 대응
- **전문가 역할**: 상담사, 교사, 기술 지원 등 특정 역할 설정
- **감정적 대응**: 공감, 격려, 안내 등 상황에 맞는 반응
- **다단계 대화**: 복잡한 문제 해결을 위한 연속적 대화

### 활용 가능한 작업 유형

| 작업 유형 | 설명 | 예시 상황 |
|----------|------|----------|
| **문서 작성** | 회의록, 보고서 등 구조화된 문서 | 업무 효율성 도구 |
| **콘텐츠 요약** | 긴 텍스트의 핵심 내용 추출 | 정보 분석 도구 |
| **텍스트 교정** | 문법, 표현, 스타일 개선 | 글쓰기 지원 도구 |
| **창작 지원** | 아이디어 발전, 스토리텔링 | 콘텐츠 제작 도구 |

In [35]:
# 대화형 Few-Shot에 사용할 다양한 작업 예시들
examples = [
    {
        "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "answer": """
회의록: XYZ 회사 마케팅 전략 회의
일시: 2023년 12월 25일
장소: XYZ 회사 회의실
참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

1. 개회
   - 회의는 김수진 팀장의 개회사로 시작됨.
   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.

2. 시장 동향 개요 (김수진)
   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.
   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.

3. 디지털 마케팅 전략 (박지민)
   - 박지민은 디지털 마케팅 전략에 대해 발표.
   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.

4. 소셜 미디어 캠페인 (이준호)
   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.
   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.

5. 종합 논의
   - 팀원들 간의 아이디어 공유 및 토론.
   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.

6. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 박지민 담당.
""",
    },
    {
        "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "answer": """
문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
- 전략: 지속 가능한 도시 개발을 달성하기 위한 다양한 전략 제시. 이에는 친환경 건축, 대중교통 개선, 에너지 효율성 증대, 지역사회 참여 강화 등이 포함됨.
- 사례 연구: 전 세계 여러 도시의 성공적인 지속 가능한 개발 사례를 소개. 예를 들어, 덴마크의 코펜하겐, 일본의 요코하마 등의 사례를 통해 실현 가능한 전략들을 설명.
- 교훈: 이러한 사례들에서 얻은 주요 교훈을 요약. 강조된 교훈에는 다각적 접근의 중요성, 지역사회와의 협력, 장기적 계획의 필요성 등이 포함됨.

이 보고서는 지속 가능한 도시 개발이 어떻게 현실적이고 효과적인 형태로 이루어질 수 있는지에 대한 심도 있는 분석을 제공합니다.
""",
    },
    {
        "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
        "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
        "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.",
    },
]

In [36]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# 채팅용 벡터 데이터베이스 생성
chroma = Chroma(
    "fewshot_chat",
    OpenAIEmbeddings(
        model="text-embedding-3-small",
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("EMBEDDING_BASE_URL"),
    ),
)

# 개별 예시를 위한 채팅 프롬프트 템플릿 (Human-AI 대화 형태)
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),  # 사용자의 지시사항과 입력
        ("ai", "{answer}"),  # AI의 응답
    ]
)

# 유사성 기반 예시 선택기 생성 (instruction 기준으로 유사성 판단)
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 선택 가능한 예시 목록
    examples,
    # 의미적 유사성을 측정하는 임베딩 모델
    embeddings,
    # 임베딩을 저장하고 유사성 검색을 수행하는 벡터스토어
    chroma,
    # 선택할 예시의 개수
    k=1,
    # 유사성 판단 기준 (instruction 필드 기준)
    input_keys=["instruction"],
)

# FewShotChatMessagePromptTemplate 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,  # 동적 예시 선택기
    example_prompt=example_prompt,  # 개별 예시 형식
)

AuthenticationError: Error code: 401 - {'error': {'message': 'Your authentication token is not from a valid issuer.', 'type': 'invalid_request_error', 'code': 'invalid_issuer', 'param': None}}

### Example Selector를 활용한 동적 예시 선택

**FewShotChatMessagePromptTemplate**과 **Example Selector**를 결합하면 사용자의 질문에 따라 가장 관련성 높은 예시가 자동으로 선택되어 프롬프트에 포함됩니다.

#### 작동 과정

| 단계 | 작업 내용 | 기술적 세부사항 |
|------|----------|---------------|
| **1. 입력 분석** | 사용자 질문의 `instruction` 필드 분석 | 임베딩 모델을 통한 벡터 변환 |
| **2. 유사성 측정** | 저장된 예시들과의 의미적 유사성 계산 | 코사인 유사도 또는 유클리드 거리 |
| **3. 예시 선택** | k개의 가장 유사한 예시 선별 | 임계값 기반 필터링 |
| **4. 프롬프트 구성** | 선택된 예시를 포함한 최적 프롬프트 생성 | 동적 템플릿 렌더링 |
| **5. 응답 생성** | 맥락에 맞는 정확한 답변 생성 | LLM을 통한 추론 실행 |

#### 설정 매개변수

```python
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,           # 선택 가능한 예시 목록
    embeddings=embeddings,       # 임베딩 모델
    vectorstore=vectorstore,     # 벡터 저장소
    k=1,                        # 선택할 예시 개수
    input_keys=["instruction"]   # 유사성 판단 기준 필드
)
```

이 방식을 통해 **맥락에 적합한 정확한 답변**을 효율적으로 생성할 수 있습니다.

In [37]:
# 회의록 작성 관련 질문으로 유사한 예시 선택 테스트
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

# instruction을 기준으로 가장 유사한 예시 선택
example_selector.select_examples(question)

NameError: name 'example_selector' is not defined

In [38]:
# 최종 채팅 프롬프트 구성: 시스템 메시지 + 예시 + 사용자 질문
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",  # 시스템 역할 정의
        ),
        few_shot_prompt,  # 선택된 예시가 자동으로 포함됨
        ("human", "{instruction}\n{input}"),  # 실제 사용자 질문
    ]
)

NameError: name 'few_shot_prompt' is not defined

In [ ]:
# 대화형 Few-Shot Chain 생성 및 실행
chain = final_prompt | llm

# 회의록 작성 요청으로 chain 실행
answer = chain.stream(question)
stream_response(answer)